# Installation

In [ ]:
# %pip install torch
# %pip install torchtext
# %pip install conllu

In [ ]:
# !unzip data.zip

# import

In [ ]:
import torch
torch.manual_seed(42) # Set Seed for reproducibility
import conllu
from io import open
from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator, Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import csv
import json
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
# from tainEmbeddings import pretrainedEmbeddings

# Get Data

In [ ]:
# Read the CoNLL-U file
def read_conllu_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = f.read()
    return data

# Parse the CoNLL-U data
def parse_conllu_data(conllu_data):
    return conllu.parse(conllu_data)

# Extract sentences and POS tags
def extract_sentences_and_postags(parsed_data):
    sentences_and_postags = []
    for tokenList in parsed_data:
        sentence = []
        postagList = []
        for token in tokenList:
            # print(token)
            word = token['form']
            postag = token['upostag']
            # filter out punctuation
            if postag != "PUNCT":
                sentence.append(word)
                postagList.append(postag)
        # if len(sentence) == 0 or len(postagList) == 0:
        #     print("Empty sentence or postagList")
        sentences_and_postags.append((sentence, postagList))
    return sentences_and_postags

def getSentencesAndPosTags(conllu_file_path):
    conllu_data = read_conllu_file(conllu_file_path)
    parsed_data = parse_conllu_data(conllu_data)
    # print(parsed_data)
    return extract_sentences_and_postags(parsed_data)

# sentences_and_postags = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-train.conllu")
# # Print the list of words and POS tags
# for sentence, postagList in sentences_and_postags:
#     print(f"sentence: {sentence}, postagList: {postagList}")


# Dataset

In [ ]:
START_TOKEN = "<s>"
END_TOKEN = "</s>"
UNKNOWN_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"
SPECIAL_TOKENS = [START_TOKEN, END_TOKEN, UNKNOWN_TOKEN, PAD_TOKEN]
UNK_POS_TAG = "UNKTAG"
SPECIAL_POS_TAG = "SPECIAL"

class PosDataset(Dataset):
    def __init__(self, data : list[tuple[list[str], list[str]]] , wordVocabulary:Vocab|None=None , tagVocabulary:Vocab|None=None):
        """Initialize the dataset. Setup Code goes here"""
        self.sentences = [i[0] for i in data] # list of sentences
        self.labels = [i[1] for i in data] # list of labels
        if wordVocabulary is None:
            sentences = [i[0] for i in data] # list of sentences
            self.wordVocabulary = build_vocab_from_iterator(sentences, specials= SPECIAL_TOKENS, min_freq= 2) # use min_freq for handling unkown words better
            self.wordVocabulary.set_default_index(self.wordVocabulary[UNKNOWN_TOKEN])
        else:
            # if vocabulary provided use that
            self.wordVocabulary = wordVocabulary
        if tagVocabulary is None:
            self.tagVocabulary = build_vocab_from_iterator(self.labels, specials= [UNK_POS_TAG,SPECIAL_POS_TAG])
            self.tagVocabulary.set_default_index(self.tagVocabulary[UNK_POS_TAG]) # index for unknown tags
        else:
            # if vocabulary provided use that
            self.tagVocabulary = tagVocabulary
        # # represent tagVocab as one hot vectors
        # self.tagVocabVectors = torch.eye(len(self.tagVocabulary))


    def __len__(self) -> int:
        """Returns number of datapoints."""
        return len(self.sentences)

    def __getitem__(self, index: int) -> tuple[torch.Tensor, torch.Tensor]:
        """Get the datapoint at `index`."""
        return torch.tensor(self.wordVocabulary.lookup_indices(self.sentences[index])), torch.tensor(self.tagVocabulary.lookup_indices(self.labels[index]))

    # def insertStartEndTokens(self, sentence: list[str]) -> list[str]:
    #     """Add start and end tokens to the sentence"""
    #     return [START_TOKEN]*self.pOfNN + sentence + [END_TOKEN]*self.sOfNN

    def createCSV(self, filename: str):
        # create a csv file with the chunks and labels to test if the dataset is created correctly
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["chunks", "labels"])
            for i in range(len(self.sentences)):
                writer.writerow([self.sentences[i], self.labels[i]])

    def collate(self, batch: list[tuple[torch.Tensor, torch.Tensor]]) -> tuple[torch.Tensor, torch.Tensor]:
        """Given a list of datapoints, batch them together"""
        sentences = [i[0] for i in batch]
        labels = [i[1] for i in batch]
        padded_sentences = pad_sequence(sentences, batch_first=True, padding_value=self.wordVocabulary[PAD_TOKEN]) # pad sentences with pad token id
        padded_labels = pad_sequence(labels, batch_first=True, padding_value= self.tagVocabulary[SPECIAL_POS_TAG]) # pad labels with special_posTag
        # padded_labels = []
        # for i in range(len(padded_sentences)):
        #     sentence = padded_sentences[i]
        #     sentencesLabels = labels[i]
        #     padded_sentence_labels = []
        #     for j in range(len(sentence)):
        #         word = sentence[j]
        #         if word == self.wordVocabulary[PAD_TOKEN]:
        #             padded_sentence_labels.append(self.tagVocabVectors[self.tagVocabulary[SPECIAL_POS_TAG]])
        #         else:
        #             padded_sentence_labels.append(sentencesLabels[j])
        #     padded_labels.append(torch.tensor(padded_sentence_labels))
        return padded_sentences, padded_labels

# LSTM Module

In [ ]:

class LSTM(torch.nn.Module):
    def __init__(self, wordVocabularySize: int, embeddingSize: int, hiddenDim: int, tagVocabSize: int, numLayers: int, classifyHiddenDim:int, activationFunction: torch.nn.Module,isBidirectional: bool) -> None:
        super().__init__()
        self.embeddingSize = embeddingSize
        self.hiddenDim = hiddenDim
        self.numLayers = numLayers
        self.wordVocabularySize = wordVocabularySize
        self.embeddingModule = torch.nn.Embedding(wordVocabularySize, embeddingSize)
        # self.embeddingModule = torch.nn.Embedding.from_pretrained(pretrainedEmbeddings)
        # self.embeddingModule.weight.requires_grad = False
        self.outputSize = tagVocabSize
        self.tagVectors = torch.eye(self.outputSize)
        self.classifyHiddenDim = classifyHiddenDim
        self.activationFunction = activationFunction
        self.isBidirectional = isBidirectional
        if isBidirectional == False:
          self.lstm = torch.nn.LSTM(self.embeddingSize,self.hiddenDim,self.numLayers)
          finalModuleList = []
          finalModuleList.append(torch.nn.Linear(self.hiddenDim,self.classifyHiddenDim))
          finalModuleList.append(self.activationFunction)
          finalModuleList.append(torch.nn.Linear(self.classifyHiddenDim,self.outputSize))
          self.finalModule = torch.nn.Sequential(*finalModuleList)
        else:
          self.lstm = torch.nn.LSTM(self.embeddingSize,self.hiddenDim,self.numLayers,bidirectional=True)
          # self.finalModule = torch.nn.Linear(self.hiddenDim*2,self.outputSize) # *2 because of bidirectional
          finalModuleList = []
          finalModuleList.append(torch.nn.Linear(self.hiddenDim*2,self.classifyHiddenDim))
          finalModuleList.append(self.activationFunction)
          finalModuleList.append(torch.nn.Linear(self.classifyHiddenDim,self.outputSize))
          self.finalModule = torch.nn.Sequential(*finalModuleList)


    def forward(self, sentence: torch.tensor):
        wordEmbeddings = self.embeddingModule(sentence)
        lstm_ouput, hid = self.lstm(wordEmbeddings.view(len(sentence), 1, -1))
        tags = self.finalModule(lstm_ouput.view(len(sentence), -1))
        tag_scores = F.log_softmax(tags,dim=1)
        return tag_scores

    def getTagVectors(self, tags: torch.tensor):
        return self.tagVectors[tags]


# HyperParameters

In [ ]:
hyperParameters0 = {
    'trainBatchSize': 1,
    'evalBatchSize': 1,
    'testBatchSize': 1,
    'learningRate': 0.3,
    'numEpochs': 10,
    'embeddingDim': 300,
    'numLayers': 2,
    'isBidirectional': True,
    'activationFunction': torch.nn.ReLU(),
    'hiddenDim': 256,
    'classifyHiddenDim': 100,
    'modelPath': "lstmConfig0.pt"
}
hyperParameters1 = {
    'trainBatchSize': 1,
    'evalBatchSize': 1,
    'testBatchSize': 1,
    'learningRate': 0.5,
    'numEpochs': 10,
    'embeddingDim': 300,
    'numLayers': 2,
    'isBidirectional': True,
    'activationFunction': torch.nn.Tanh(),
    'hiddenDim': 256,
    'classifyHiddenDim': 100,
    'modelPath': "lstmConfig1.pt"
}
hyperParameters2 = {
    'trainBatchSize': 1,
    'evalBatchSize': 1,
    'testBatchSize': 1,
    'learningRate': 1e-2,
    'numEpochs': 10,
    'embeddingDim': 300,
    'numLayers': 2,
    'isBidirectional': False,
    'activationFunction': torch.nn.Tanh(),
    'hiddenDim': 256,
    'classifyHiddenDim': 100,
    'modelPath': "lstmConfig2.pt"
}
hyperParametersList = [hyperParameters0,hyperParameters1,hyperParameters2]
hyperParameters = None
epochLists = []
devAccLists = []
for config in hyperParametersList:
  epochNums = []
  devAccuracies = []
  hyperParameters = config
  # # load word , tag vocabularies
  # wordVocab = torch.load("trainWordVocab.pth")
  # tagVocab = torch.load("trainTagVocab.pth")
  # print(type(wordVocab))

  trainRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-train.conllu")
  evalRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-dev.conllu")
  testRawData = getSentencesAndPosTags("./data/UD_English-Atis/en_atis-ud-test.conllu")
  trainDataset = PosDataset(trainRawData)
  wordVocab = trainDataset.wordVocabulary
  tagVocab = trainDataset.tagVocabulary
  evalDataset = PosDataset(evalRawData , wordVocabulary=wordVocab, tagVocabulary=wordVocab)
  testDataset = PosDataset(testRawData , wordVocabulary=wordVocab, tagVocabulary=wordVocab)
  trainDataLoader = DataLoader(trainDataset,batch_size= hyperParameters['trainBatchSize'] , shuffle=True, collate_fn=trainDataset.collate)
  evalDataLoader = DataLoader(evalDataset,batch_size= hyperParameters['trainBatchSize'],  collate_fn= evalDataset.collate) # dont shuffle here for consistent evaluation
  testDataLoader = DataLoader(testDataset, batch_size= hyperParameters['trainBatchSize'], collate_fn= testDataset.collate) # dont shuffle here for consistent evaluation

  # # create train.csv, eval.csv, test.csv
  # trainDataset.createCSV("lstmtrain.csv")
  # evalDataset.createCSV("lstmeval.csv")
  # testDataset.createCSV("lstmtest.csv")
  # # store vocabularies

  print(trainDataset[7])
  # for batch, (sentences, tags) in enumerate(trainDataLoader):
  #     print(sentences)
  #     print(tags)
  #     break
  trainWordVocabSize = len(wordVocab)
  trainTagVocabSize = len(tagVocab)
  myPosTagger = LSTM(trainWordVocabSize,hyperParameters['embeddingDim'], hyperParameters['hiddenDim'],trainTagVocabSize,hyperParameters['numLayers'],hyperParameters['classifyHiddenDim'],hyperParameters['activationFunction'], hyperParameters['isBidirectional'])
  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(device)
  # lossFn = torch.nn.CrossEntropyLoss(ignore_index= tagVocab[SPECIAL_POS_TAG]) # ignore padding index

  lossFn = torch.nn.NLLLoss(ignore_index= tagVocab[SPECIAL_POS_TAG]) # ignore padding index
  optimizer = torch.optim.Adam(myPosTagger.parameters(), lr=hyperParameters['learningRate'])


  # Send Model to  Device
  myPosTagger = myPosTagger.to(device)

  epochNums = []
  devAccuracies = []
  for epochNum in range(hyperParameters['numEpochs']):
      # Set to Training Mode
      myPosTagger.train()
      batchNum = 0
      for (inputSentences , tagsIndices) in trainDataLoader:
        # with torch.device(device):
          # for batchsize = 1
          inputSentences = inputSentences[0]
          tagsIndices = tagsIndices[0]
          # Zero the gradients
          optimizer.zero_grad()
          # # Convert to one-hot encoding
          # tagsList = myPosTagger.tagVectors[tagsIndices]
          tagsList = tagsIndices
          # Send data to device
          inputSentences = inputSentences.to(device)
          tagsList = tagsList.to(device)
          # Forward Pass
          pred = myPosTagger(inputSentences)
          # print(pred.shape)
          # print(tagsList.shape)
          loss = lossFn(pred, tagsList)
          # Backward Pass
          loss.backward()
          optimizer.step()
          # Print Loss after every 50 batches
          batchNum += 1
          if batchNum % 500 == 0:
              print(f'Epoch: {epochNum}, Batch: {batchNum}, Loss: {loss.item()}')

      # Set to Evaluation Mode
      myPosTagger.eval()
      evalLoss = 0
      correct = 0
      total = 0
      # turn off gradients
      with torch.no_grad():
          for  (inputSentences , tagsIndices) in evalDataLoader:
              # for batchsize = 1
              inputSentences = inputSentences[0]
              tagsIndices = tagsIndices[0]
              # # Convert to one-hot encoding
              # tagsList = myPosTagger.tagVectors[tagsIndices]
              tagsList = tagsIndices
              # Send data to device
              inputSentences = inputSentences.to(device)
              tagsList = tagsList.to(device)
              # Forward Pass
              pred = myPosTagger(inputSentences)
              loss = lossFn(pred, tagsList)
              evalLoss += loss.item()
              # Get Predictions
              pred = torch.argmax(pred, dim=1)
              # tagIndices = torch.argmax(tagsList, dim=1)
              tagIndices = tagsList
              # # total number of words in all the sentences
              # total += len(tagIndices)*len(tagIndices[0])
              # # number of correct predictions
              # for i in range(len(tagIndices)):
              #     for j in range(len(tagIndices[i])):
              #         if tagIndices[i][j] == pred[i][j]:
              #             correct += 1
              total += len(tagIndices)
              correct += (pred == tagIndices).sum().item()
      # Print validation Loss and Accuracy
      print(f'Epoch: {epochNum}, validation Loss: {evalLoss/len(evalDataLoader)}, Val Accuracy: {(correct/total)*100} %')
      epochNums.append(epochNum)
      devAccuracies.append((correct/total)*100)
  # save model and hyper parameters
  torch.save(myPosTagger, hyperParameters['modelPath'])
  # with open('hyperparameters1.json', 'w') as f:
  #     json.dump(hyperParameters, f)
  loadedPosTagger = torch.load(hyperParameters['modelPath'])
  # move the model to the GPU
  loadedPosTagger = loadedPosTagger.to(device)
  # set the testing model to the evaluation mode
  loadedPosTagger.eval()
  lossFn = torch.nn.NLLLoss(ignore_index = tagVocab[SPECIAL_POS_TAG]) # ignore padding index
  testLoss = 0
  correct = 0
  total = 0
  # turn off gradients
  with torch.no_grad():
      for batchNum, (inputSentences , tagsIndices) in enumerate(testDataLoader):
          # for batchsize = 1
          inputSentences = inputSentences[0]
          tagsIndices = tagsIndices[0]
          tagsList = tagsIndices
          # Send data to device
          inputSentences = inputSentences.to(device)
          tagsList = tagsList.to(device)
          # Forward Pass
          pred = loadedPosTagger(inputSentences)
          loss = lossFn(pred, tagsList)
          testLoss += loss.item()
          # Get Predictions
          pred = torch.argmax(pred, dim=1)
          # tagIndices = torch.argmax(tagsList, dim=1)
          tagIndices = tagsList
          total += tagIndices.size(0)
          correct += (pred == tagIndices).sum().item()
  # Print test Loss and Accuracy
  print(f'test Loss: {testLoss/len(testDataLoader)}, Test Accuracy: {(correct/total)*100} %')
  epochLists.append(epochNums)
  devAccLists.append(devAccuracies)


# plot epoch vs dev set accuracy

In [ ]:
epochNums = epochLists[0]
devAccuracies = devAccLists[0]
plt.plot(epochNums,devAccuracies, label = 'config0')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.show()


In [ ]:
epochNums = epochLists[1]
devAccuracies = devAccLists[1]
plt.plot(epochNums,devAccuracies, label = 'config1')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.show()

In [ ]:
epochNums = epochLists[2]
devAccuracies = devAccLists[2]
plt.plot(epochNums,devAccuracies, label = 'config2')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.show()